In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import *
from sklearn.metrics import roc_auc_score, log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import optuna

In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

def preprocessing(train, test):
    
    # 1. 필요 없는 컬럼 제거
    non_imp_cols = ['난자 혼합 경과일',
    '기증자 정자와 혼합된 난자 수',
    '동결 배아 사용 여부',
    '여성 주 불임 원인',
    '불명확 불임 원인',
    '부부 부 불임 원인',
    '착상 전 유전 진단 사용 여부',
    '시술 유형',
    '남성 부 불임 원인',
    '기증 배아 사용 여부',
    '불임 원인 - 정자 운동성',
    '남성 주 불임 원인',
    '불임 원인 - 자궁내막증',
    '불임 원인 - 정자 농도',
    '여성 부 불임 원인',
    '대리모 여부',
    '불임 원인 - 정자 면역학적 요인',
    '저장된 신선 난자 수',
    '부부 주 불임 원인',
    '착상 전 유전 검사 사용 여부',
    '불임 원인 - 여성 요인',
    '불임 원인 - 자궁경부 문제',
    'PGD 시술 여부',
    'PGS 시술 여부',
    '난자 채취 경과일',
    '난자 해동 경과일',
    '불임 원인 - 정자 형태']
    # '착상 전 유전 검사 사용 여부',
    # '착상 전 유전 진단 사용 여부',
    # '남성 주 불임 원인',
    # '남성 부 불임 원인',
    # '여성 주 불임 원인',
    # '여성 부 불임 원인',
    # '부부 주 불임 원인',
    # '부부 부 불임 원인',
    # '불명확 불임 원인',
    # '불임 원인 - 난관 질환',
    # '불임 원인 - 남성 요인',
    # '불임 원인 - 배란 장애',
    # '불임 원인 - 여성 요인',
    # '불임 원인 - 자궁경부 문제',
    # '불임 원인 - 자궁내막증',
    # '불임 원인 - 정자 농도',
    # '불임 원인 - 정자 면역학적 요인',
    # '불임 원인 - 정자 운동성',
    # '불임 원인 - 정자 형태',
    # 'IVF 시술 횟수',
    # 'DI 시술 횟수',
    # '총 임신 횟수',
    # 'IVF 임신 횟수',
    # 'DI 임신 횟수',
    # '총 출산 횟수',
    # 'IVF 출산 횟수',
    # 'DI 출산 횟수',
    # 'PGD 시술 여부',
    # 'PGS 시술 여부',
    # '대리모 여부',
    # '난자 채취 경과일',
    # '배란 유도 유형',
    # '정자 기증자 나이']
    
    train.drop(columns=non_imp_cols, inplace=True)
    test.drop(columns=non_imp_cols, inplace=True)

    # 미확인 원소 대체
    train['시술 당시 나이'] = train['시술 당시 나이'].replace({'알 수 없음' : None})
    test['시술 당시 나이'] = test['시술 당시 나이'].replace({'알 수 없음' : None})
    train['특정 시술 유형'] = train['특정 시술 유형'].replace({'Unknown' : 'IVF'})
    test['특정 시술 유형'] = test['특정 시술 유형'].replace({'Unknown' : 'IVF'})
    train['난자 출처'] = train['난자 출처'].replace({'알 수 없음' : '본인 제공'})
    test['난자 출처'] = test['난자 출처'].replace({'알 수 없음' : '본인 제공'})

    # 결측치 최빈값 대체
    train = train.apply(lambda x:x.fillna(x.mode()[0]))
    test = test.apply(lambda x:x.fillna(x.mode()[0]))

    categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
    ]
    categorical_columns = [x for x in categorical_columns if x not in non_imp_cols]
    
    for col in categorical_columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)


    return train, test, categorical_columns

train, test, categorical_columns = preprocessing(train, test)

In [5]:
# from sklearn.preprocessing import QuantileTransformer

# numeric_cols = X.select_dtypes(include=[np.number]).columns
# qt = QuantileTransformer(output_distribution='normal', random_state=42)
# qt = QuantileTransformer(output_distribution='uniform', random_state=42)
# X[numeric_cols] = qt.fit_transform(X[numeric_cols])
# test[numeric_cols] = qt.fit_transform(test[numeric_cols])

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
# test[numeric_cols] = scaler.fit_transform(test[numeric_cols])


### 컬럼 조작


In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:

# X_train_numeric = X_train[numeric_cols]
# X_valid_numeric = X_valid[numeric_cols]
# test_numeric = test[numeric_cols]

X_train_categoric = X_train[categorical_columns]
X_valid_categoric = X_valid[categorical_columns]
test_categoricric = test[categorical_columns]


In [9]:
import pickle
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm  # 진행바 라이브러리

# 'model.pkl' 파일에 저장된 모델 파라미터를 불러오기
with open('./best_params_cat_data_drop2.pkl', 'rb') as f:
    params = pickle.load(f)

n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 3. 교차검증 수행 및 결과 저장
log_loss_scores = []
roc_scores = []
models = []  # 각 Fold별 모델 저장

for fold, (train_idx, val_idx) in enumerate(tqdm(skf.split(X_train, y_train), total=n_splits, desc="Fold 진행")):
    print(f"\n📌 Fold {fold+1}/{n_splits} 시작...")
    
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # CatBoost 모델 설정
    model = CatBoostClassifier(
        **params,
        loss_function='Logloss',
        verbose=0,
        cat_features=categorical_columns,
        random_state=42
    )
    
    # 모델 학습
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=100)
    
    # 검증 데이터에 대한 예측 및 Log Loss 계산
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    loss = log_loss(y_val, y_pred_proba)
    roc = roc_auc_score(y_val, y_pred_proba)
    log_loss_scores.append(loss)
    roc_scores.append(roc)
    
    # 모델 저장
    models.append(model)
    print(f"✔ Fold {fold+1} Log Loss: {loss:.4f}")

print("\n평균 검증 ROC-AUC: {:.4f}".format(np.mean(roc_scores)))
print("\n평균 검증 Loss: {:.4f}".format(np.mean(log_loss_scores)))

# 4. 앙상블: 테스트 데이터에 대해 각 Fold 모델의 예측 확률 평균
ensemble_preds = np.zeros(X_valid.shape[0])
for model in models:
    ensemble_preds += model.predict_proba(X_valid)[:, 1]
ensemble_preds /= len(models)

# 5. 최종 테스트 데이터 ROC-AUC 평가
ensemble_auc = roc_auc_score(y_valid, ensemble_preds)
print("앙상블 모델의 테스트 ROC-AUC: {:.4f}".format(ensemble_auc))

"""
standard scaler
평균 검증 ROC-AUC: 0.7392

평균 검증 Loss: 0.4889

앙상블 모델의 테스트 ROC-AUC: 0.7390

qunatile transformer
평균 검증 ROC-AUC: 0.7389

평균 검증 Loss: 0.4892

앙상블 모델의 테스트 ROC-AUC: 0.7391
"""

Fold 진행:   0%|          | 0/3 [00:00<?, ?it/s]


📌 Fold 1/3 시작...
0:	learn: 0.6120678	test: 0.6123170	best: 0.6123170 (0)	total: 242ms	remaining: 1m 19s
100:	learn: 0.4850682	test: 0.4892985	best: 0.4892750 (98)	total: 9.97s	remaining: 22.6s
200:	learn: 0.4803915	test: 0.4891353	best: 0.4890280 (164)	total: 20.1s	remaining: 12.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4890280042
bestIteration = 164

Shrink model to first 165 iterations.


Fold 진행:  33%|███▎      | 1/3 [00:23<00:47, 23.90s/it]

✔ Fold 1 Log Loss: 0.4890

📌 Fold 2/3 시작...
0:	learn: 0.6140371	test: 0.6137921	best: 0.6137921 (0)	total: 217ms	remaining: 1m 11s
100:	learn: 0.4845792	test: 0.4891695	best: 0.4891685 (99)	total: 9.99s	remaining: 22.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4887539603
bestIteration = 131

Shrink model to first 132 iterations.


Fold 진행:  67%|██████▋   | 2/3 [00:44<00:22, 22.22s/it]

✔ Fold 2 Log Loss: 0.4888

📌 Fold 3/3 시작...
0:	learn: 0.6116527	test: 0.6118391	best: 0.6118391 (0)	total: 140ms	remaining: 46.2s
100:	learn: 0.4844820	test: 0.4902900	best: 0.4902900 (100)	total: 9.52s	remaining: 21.6s
200:	learn: 0.4790675	test: 0.4899879	best: 0.4898964 (193)	total: 20s	remaining: 12.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4898963807
bestIteration = 193

Shrink model to first 194 iterations.


Fold 진행: 100%|██████████| 3/3 [01:12<00:00, 24.27s/it]

✔ Fold 3 Log Loss: 0.4899

평균 검증 ROC-AUC: 0.7389

평균 검증 Loss: 0.4892


앙상블 모델의 테스트 ROC-AUC: 0.7391


'\nstandard scaler\n평균 검증 ROC-AUC: 0.7392\n\n평균 검증 Loss: 0.4889\n\n앙상블 모델의 테스트 ROC-AUC: 0.7390\n\n\n'

In [10]:
ensemble_preds = np.zeros(test.shape[0])
for model in models:
    ensemble_preds += model.predict_proba(test)[:, 1]
ensemble_preds /= len(models)

print("예측 확률:", ensemble_preds)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = ensemble_preds
sample_submission.to_csv('./submit/submit_26.csv', index=False)

예측 확률: [0.00818519 0.00298527 0.15506806 ... 0.43729072 0.23041865 0.00855836]


In [6]:
import pickle

# 'model.pkl' 파일에 저장된 모델 파라미터를 불러오기
with open('./best_params_cat_data_drop2.pkl', 'rb') as f:
    params = pickle.load(f)

model = CatBoostClassifier(
        **params,
        loss_function='Logloss',
        verbose=0,
        cat_features=categorical_columns,
        random_state=42
)

model.fit(
        X_train_categoric, y_train,
        eval_set=(X_valid_categoric, y_valid),
        early_stopping_rounds=20
)

importances = model.get_feature_importance()
feature_names = X_train_categoric.columns
fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)


predictions = model.predict_proba(X_valid_categoric)[:, 1]
print(roc_auc_score(y_valid, predictions))
# 0.7393

0.6675506834780642


In [6]:
# 기본
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] == 0]
display(df_new['feature'].tolist())

,feature,importance
35,이식된 배아 수,74.816914
1,시술 당시 나이,4.866493
37,저장된 배아 수,2.244027
54,배아 이식 경과일,2.100530
32,총 생성 배아 수,1.928422
41,수집된 신선 난자 수,1.192106
4,특정 시술 유형,0.997900
43,혼합된 난자 수,0.851883
48,난자 기증자 나이,0.807193
23,클리닉 내 총 시술 횟수,0.775755


[]

In [8]:
# 기본
# display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

,feature,importance
35,이식된 배아 수,74.816914
1,시술 당시 나이,4.866493
37,저장된 배아 수,2.244027
54,배아 이식 경과일,2.100530
32,총 생성 배아 수,1.928422
41,수집된 신선 난자 수,1.192106
4,특정 시술 유형,0.997900
43,혼합된 난자 수,0.851883
48,난자 기증자 나이,0.807193
23,클리닉 내 총 시술 횟수,0.775755


In [10]:
# standard scaler
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

,feature,importance
41,이식된 배아 수,80.680642
1,시술 당시 나이,4.106672
38,총 생성 배아 수,2.839445
27,배아 생성 주요 이유,2.529968
65,배아 이식 경과일,1.778122
...,...,...
61,PGS 시술 여부,0.000000
62,난자 채취 경과일,0.000000
63,난자 해동 경과일,0.000000
20,불임 원인 - 여성 요인,0.000000


,feature,importance
41,이식된 배아 수,80.680642
1,시술 당시 나이,4.106672
38,총 생성 배아 수,2.839445
27,배아 생성 주요 이유,2.529968
65,배아 이식 경과일,1.778122
43,저장된 배아 수,1.494529
47,수집된 신선 난자 수,1.130083
52,난자 출처,0.808021
6,배란 유도 유형,0.674208
54,난자 기증자 나이,0.494848


In [10]:
# quantile transform
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

,feature,importance
41,이식된 배아 수,71.225708
43,저장된 배아 수,6.336121
1,시술 당시 나이,5.858333
38,총 생성 배아 수,2.686949
65,배아 이식 경과일,1.946295
...,...,...
61,PGS 시술 여부,0.000000
62,난자 채취 경과일,0.000000
63,난자 해동 경과일,0.000000
23,불임 원인 - 정자 농도,0.000000


,feature,importance
41,이식된 배아 수,71.225708
43,저장된 배아 수,6.336121
1,시술 당시 나이,5.858333
38,총 생성 배아 수,2.686949
65,배아 이식 경과일,1.946295
52,난자 출처,1.381438
6,배란 유도 유형,1.257522
47,수집된 신선 난자 수,1.226122
27,배아 생성 주요 이유,0.923097
17,불임 원인 - 난관 질환,0.561220


In [10]:
# uniform transform
display(fi_df)
df_new = fi_df.loc[fi_df['importance'] != 0]
display(df_new)

,feature,importance
41,이식된 배아 수,70.307673
1,시술 당시 나이,5.073480
38,총 생성 배아 수,3.009302
65,배아 이식 경과일,2.351401
43,저장된 배아 수,2.317679
...,...,...
61,PGS 시술 여부,0.000000
62,난자 채취 경과일,0.000000
63,난자 해동 경과일,0.000000
20,불임 원인 - 여성 요인,0.000000


,feature,importance
41,이식된 배아 수,70.307673
1,시술 당시 나이,5.073480
38,총 생성 배아 수,3.009302
65,배아 이식 경과일,2.351401
43,저장된 배아 수,2.317679
0,시술 시기 코드,1.383140
27,배아 생성 주요 이유,1.224158
29,클리닉 내 총 시술 횟수,1.163463
30,IVF 시술 횟수,1.048123
52,난자 출처,0.947658
